In [1]:
cd ..

/home/amiyaguchi/cs224w/wikipedia-retention


In [2]:
from pyspark.sql import SparkSession, functions as F, types as T

spark = SparkSession.builder.getOrCreate()
enmain = spark.read.parquet("data/interim/enwiki-meta-main")
enmain.createOrReplaceTempView("enmain")

In [4]:
article_sample = set(
    map(lambda x: x.article_id, (
            enmain
            .groupBy("article_id")
            .agg(F.count("*").alias("n_edits"))
            .where("n_edits > 10 and n_edits < 50")
            .orderBy(F.rand())
            .take(1000)
        )))

In [10]:
from pyspark.sql.window import Window

previous_revision = Window.partitionBy("article_id").orderBy("rev_id")

rev_diffs = (
    enmain
    .select(
        "user_id",
        "user_name",
        "article_id",
        "article_title",
        "textdata",
        "rev_id",
        "timestamp",
        (F.size("main") - F.lag(F.size("main")).over(previous_revision)).alias("link_diff"),
        F.array_except(F.lag("main").over(previous_revision), "main").alias("link_remove"),
        F.array_except("main", F.lag("main").over(previous_revision)).alias("link_add")
    )
    .where("link_diff <> 0")
    .orderBy("article_title", "rev_id")
)

rev_diffs.createOrReplaceTempView("rev_diffs")

AnalysisException: "cannot resolve '`user_name`' given input columns: [timestamp, username, main, category, user_talk, minor, external, other, year, comment, article_title, talk, user_id, user, template, textdata, article_id, quarter, image, rev_id];;\n'Project [user_id#5, 'user_name, article_id#0, article_title#2, textdata#17, rev_id#1, timestamp#3, (size(main#8) - lag(size(main#8), 1, null) windowspecdefinition(article_id#0, rev_id#1 ASC NULLS FIRST, specifiedwindowframe(RowFrame, -1, -1))) AS link_diff#40696, array_except(lag(main#8, 1, null) windowspecdefinition(article_id#0, rev_id#1 ASC NULLS FIRST, specifiedwindowframe(RowFrame, -1, -1)), main#8) AS link_remove#40697, array_except(main#8, lag(main#8, 1, null) windowspecdefinition(article_id#0, rev_id#1 ASC NULLS FIRST, specifiedwindowframe(RowFrame, -1, -1))) AS link_add#40698]\n+- Relation[article_id#0,rev_id#1,article_title#2,timestamp#3,username#4,user_id#5,category#6,image#7,main#8,talk#9,user#10,user_talk#11,other#12,extern

In [9]:
rev_diffs.count()

32453659

In [ ]:
link_from = rev_diffs.withColumn("src", F.explode("link_add"))
link_to = rev_diffs.withColumn("dst", F.explode("link_remove"))

link_from.alias("link_from")
link_to.alias("link_to")

edges = spark.sql("""
-- why doesnt left outer join work?

SELECT
    s.user_id,
    s.user_name,
    s.article_id,
    s.rev_id,
    s.timestamp as added_ts,
    t.timestamp as removed_ts,
    t.rev_id - s.rev_id as span_rev,
    (unix_timestamp(t.timestamp) - unix_timestamp(s.timestamp)) as span_seconds,
    s.article_title as from,
    t.to
FROM link_from AS s
JOIN link_to AS t
ON s.article_title = t.article_title
    AND s.from = t.to
    AND s.rev_id < t.rev_id
    AND s.timestamp < t.timestamp

UNION

SELECT
    s.user_id,
    s.user_name,
    s.article_id,
    s.rev_id,
    s.timestamp as added_ts,
    null as removed_ts,
    null as span_rev,
    null as span_seconds,
    s.article_title as from,
    null as to
FROM link_from AS s
LEFT OUTER JOIN link_to AS t
ON s.article_title = t.article_title
    AND t.to is null
    AND s.rev_id < t.rev_id
    AND s.timestamp < t.timestamp
""")

edges.orderBy("from", "rev_id").write("processed/rev_history/v1")

In [ ]:
edges = spark.read.parquet("processed/rev_history/v1")
edges.show()

In [ ]:
%time edges.count()

In [ ]:
edges.where("to is null").count()